## Machine Learning Record Mining

Project to create a pipeline that uses GeoDeepDive's output to find Unaquired Sites for Neotoma.

Using NLP parsed text and a Data Science approach, identify whether a paper is suitable for Neotoma and detect features such as 'Site Name', 'Location', 'Age Span' and 'Site Descriptions'.

In [1]:
# Loading libraries

import numpy as np
import pandas as pd
import csv
import psycopg2
import re
import ast
import os

# Load Postgres Server
from src.config import config

In [2]:
# Options for DF display
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 10)

## Loading and viewing the Data

### Loading NLP Sentences

In [4]:
# Loading from script's output

nlp_sentences = pd.read_csv('src/output/nlp_sentences_df.csv', delimiter='\t')

In [5]:
nlp_sentences.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified,words_as_string,dms_regex,dd_regex
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['Available', 'online', 'at', 'www.sciencedire...","['JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'CD', ...","['O', 'O', 'O', 'O', 'O', 'O', 'NUMBER', 'O', ...","['available', 'online', 'at', 'www.sciencedire...","['dep', 'dep', 'dep', 'dep', 'dep', 'dep', 'de...","[218, 218, 218, 218, 218, 218, 218, 0, 218, 0,...","Available,online,at,www.sciencedirect.com,Quat...",[],[]
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","['The', 'Chihuahueños', 'Bog', 'record', 'exte...","['DT', 'NNP', 'NN', 'NN', 'VBZ', 'TO', 'IN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'NUMBER', ...","['the', 'Chihuahueños', 'bog', 'record', 'exte...","['det', 'compound', 'compound', 'nsubj', '', '...","[4, 4, 4, 5, 0, 11, 11, 11, 11, 11, 5, 0]","The,Chihuahueños,Bog,record,extends,to,over,15...",[],[]
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['An', 'Artemisia', 'steppe', ',', 'then', 'an...","['DT', 'NNP', 'NN', ',', 'RB', 'DT', 'JJ', 'NN...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION'...","['a', 'Artemisia', 'steppe', ',', 'then', 'a',...","['det', 'compound', 'nsubj', '', 'advmod', 'de...","[3, 3, 10, 0, 9, 9, 9, 9, 10, 0, 14, 14, 14, 1...","An,Artemisia,steppe,,,then,an,open,Picea,woodl...",[],[]


In [6]:
# Running this takes forever. Do not run unless necessary
# nlp_sentences.describe(include = 'all')

In [7]:
avg_sentences = nlp_sentences.groupby('_gddid')['_gddid','sentid'].transform('max')
avg_sentences.drop_duplicates(subset='_gddid', keep='first', inplace=True)
avg_sentences.describe()

,sentid
count,218.000000
mean,513.600917
std,232.460891
min,2.000000
25%,397.250000
50%,504.000000
75%,638.250000
max,1330.000000


From the Summary Statistics, despite having 111,965 sentences, there are only 218 unique articles (`_gddid`). The article with most sentences, had 1330 sentences; that's the max number found in `sentid`.  
In order to know the average of sentences, I grouped by `_gddid` and then, obtained the number of sentences per article. After that, I just got the mean through a Summary of statistics to know that on average, each article has 514 sentences.

### Loading Bibliography Data

In [8]:
bibliography = pd.read_csv('src/output/bibliography.csv', delimiter=',')
bibliography.head(3)

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
0,doi,10.1016/S0277-3791(99)00007-4,Elsevier,"Palaeoclimate, chronology and vegetation histo...",Quaternary Science Reviews,"Huntley, B.; Watts, W.A.; Allen, J.R.M.; Zolit...",1999,7,NaN,550453fde1382326932d85f7,article,945--960,http://www.sciencedirect.com/science/article/p...,publisher
1,doi,10.1139/b92-002,Canadian Science Publishing,Holocene history of forest trees on the Bruce ...,Canadian Journal of Botany,"Bennett, K. D.",1992,1,NaN,578b5aabcf58f1587003efba,article,6--18,http://www.nrcresearchpress.com/doi/abs/10.113...,publisher
2,doi,10.1016/0031-0182(80)90043-7,Elsevier,Glacial sequence and environmental history in ...,"Palaeogeography, Palaeoclimatology, Palaeoecology","Van Der Hammen, T.; Barelds, J.; De Jong, H.; ...",1980,NaN,NaN,54b43244e138239d868493cd,article,247--340,http://www.sciencedirect.com/science/article/p...,publisher


In [9]:
bibliography.describe(include = 'all')

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
count,221,221,221,221,221,221,221.000000,183,0.0,221,221,216,220,220
unique,1,221,6,221,41,213,NaN,28,NaN,221,1,214,220,1
top,doi,10.1016/j.yqres.2008.03.006,Elsevier,Spatio-temporal patterns in Lateglacial and Ho...,Quaternary Research,"Markgraf, Vera",NaN,3,NaN,578d37a2cf58f1101ff90643,article,1--13,http://doi.wiley.com/10.1046/j.1365-2699.2002....,publisher
freq,221,1,138,1,44,3,NaN,40,NaN,1,221,2,1,220
mean,NaN,NaN,NaN,NaN,NaN,NaN,2001.438914,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
min,NaN,NaN,NaN,NaN,NaN,NaN,1963.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,1995.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,2005.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,2011.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EDA

Reviewing our data includes skimming through some papers online and seeing if the data is consistent with our NLP Sentences dataframe.

From there, we can also visualize what we would like our model to predict: 'Location', 'Site Name', 'Age Span', and 'Site Description' from a "Human perspective".

### Finding Latitude and Longitud via REGEX 

After reviewing some PDF's and viewing the data parsing, I noticed some possible patterns for coordinates.

<img src="figures/img/00_snippet_of_pdf.png" alt="Drawing" style="width: 300px;"/>

Which translates to our pandas dataframe as:

![alt text](figures/img/01_snippet_in_sql.png "PandasVersion")

As a first problem, we can notice that the degrees symbol `°` was translated as a `3`. On some other occassions, it is confused as an `o` or a middle point `◦`. For `'`, this symbol got confused by the word `cents`.

I am still going to use normal REGEX to find coordinates in the most intuitive way. 

A coordinate for longitude is a number ranging from -180° to 180°. It has three components, `°, ',"` and it covers the position in E or W.

Latitude goes from -90° to 90°, has the same three components and goes from S to N.

This is the used REGEX to extract coordinates in the script:

`dms_regex = r"([-]?\d{1,3}[°|′|\'|,]{0,3}\d{1,2}[,|′|\']{1,3}\d{0,2}[,|′|\']{1,4}[NESWnesw]?[\s|,|\']+?[-]?\d{1,3}[°|,|′|\']+\d{1,2}[,|′|\']+\d{0,2}[,|′|\'][,|′|\']{0,4}[NESWnesw]?)"`

`dd_regex =  r"([-]?\d{1,3}\.\d{1,}[,]?[NESWnesw][\s|,|\']+?[-]?\d{1,3}\.\d{1,}[,]?[NESWnesw])"`


In [10]:
# Run only when enough time available
# nlp_sentences[['words_as_string', 'words', 'dms_regex', 'dd_regex']].describe()

The `dms_regex` only returned 95 unique values. That's to say that from the 111965 sentences, only 95 possess something similar to the regex we are looking for.  However, the `dd_regex` returned 8 values.

Let's see an example of each regex extraction to verify values.

In [11]:
# dms regex example
dms_example = nlp_sentences[nlp_sentences['_gddid'] == '550453fde1382326932d85f7']
dms_example = dms_example[dms_example['sentid'] == 10]
dms_example[['words', 'words_as_string', 'dms_regex', 'dd_regex']]

,words,words_as_string,dms_regex,dd_regex
94875,"['Introduction', 'Laghi', 'di', 'Monticchio', ...","Introduction,Laghi,di,Monticchio,are,two,maar,...","['403,56,40,N,,,153,36,48,E']",[]


In [12]:
# dd_regex example
dd_example = nlp_sentences[nlp_sentences['_gddid'] == '57a12aaccf58f1b0962a836c']
dd_example = dd_example[dd_example['sentid'] == 62]
dd_example[['words', 'words_as_string', 'dms_regex', 'dd_regex']]

,words,words_as_string,dms_regex,dd_regex
62439,"['Kingston', 'meadow', 'Kingston', 'Meadow', '...","Kingston,meadow,Kingston,Meadow,-LRB-,2400,m,e...",[],"['117.11,W,39.21,N']"


### Problems with the REGEX approach

As seen before, extracting information using REGEX is very complicated. We would require a lot of queries that would retrieve each particular case.

A possible alternative could be using NER, Named Entity Recognition.

In [13]:
def order_article(article_id):
    '''
    Function to find an article by its gddid in the NLP sentences and have it displayed in order
    
    Keyword arguments:
    article_id -- gddid
    
    Returns:
    article ordered by sentence index
    '''
    article = nlp_sentences[nlp_sentences['_gddid'] == article_id]
    return article[['sentid', 'words']].sort_values(by = 'sentid')

Article 01  
Skimmed info: 
 * `Article Name:` Palaeoclimate_chronology_and_vegetation_history_of_the_Weichselian_Lateglacial
 * `Link`: http://www.sciencedirect.com/science/article/pii/S0277379199000074  
 * `Site Name`:  Laghi di Monticchio
 * `Location`:  40° 56' 40" N, 15° 36' 48" E
 * `Age Span`: 
 * `Site Descriptions`: The record from Lago Grande di Monticchio now adds macrofossil evidence of the presence of relatively boreal tree taxa during the Lateglacial interstadial and provides a chronology based upon annually laminated sediments.

In [14]:
# Article 01 
order_article('550453fde1382326932d85f7')

,sentid,words
94865,1,"['Quaternary', 'Science', 'Reviews', '18', '-L..."
94866,2,"['Plant', 'macrofossil', 'data', 'from', 'one'..."
94867,3,"['Quantitative', 'palaeoclimate', 'reconstruct..."
94868,4,"['The', 'chronological', 'framework', 'for', '..."
94870,5,"['Surface', 'conditions', 'in', 'the', 'North'..."
...,...,...
95325,448,"['Vegetation', 'history', 'and', 'climate', 'o..."
95326,449,"['Quaternary', 'Science', 'Reviews', ',', '15'..."
95327,450,"['Zolitschka', ',', 'B.', ',', '&', 'Negendank..."
95329,451,"['Sedimentology', ',', 'dating', 'and', 'palae..."


Article 02  
Skimmed info:  
* `Article:` Development of the Mixed Connifer Forest in Northern New Mexico and its relationship to Holocene environmental change
* `Link`: http://www.sciencedirect.com/science/article/pii/S0033589407001512
* `Site Name`:  Chihuahuenos Bog 
* `Location`:  ??? No Given Coordinates 
* `Age Span`:   
* `Site Descriptions`:  Chihuahueños Bog is located within the mixed conifer forest and has affinities to the flora of the southern Rocky Mountains(Foxx and Tierney, 1985; Allen, 2004). Although the area immediately surrounding the bog has been heavily logged in recent years, nearly every conifer species growing in montaneforests of the Jemez mountains is represented either along theshore of the bog or in the surrounding uplands.  

In [15]:
# Article 2
# order_article('54b43266e138239d8684efed')

Article 03  
Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/g35541.1  # No Full access to article
* `Site Name`:  Cahokia region, Illinois, USA
* `Location`:  N38°42'00" - N38°42'00", W90°04'60" - W90°04'60" - Not found on text (yet).
* `Age Span`:   
* `Site Descriptions`: Record from Horseshoe Lake, an oxbow lake in the central Mississippi River valley that is adjacent to the Cahokia site (Illinois, USA), the largest prehistoric settlement north of Mexico.   

In [16]:
# Article 3
# order_article('57c5b941cf58f1338eaddb5b')

Article 04  
Skimmed info:  
* `Link:`http://www.tandfonline.com/doi/abs/10.1080/00173130902965157
* `Site Name`:  Begbunar
* `Location`:  42°09′ N, 22° 33′ E; 1750 ; in SQL file shows as: `42,°,09,cents,N,,,22,°,33,cents,E`
* `Age Span`:   
* `Site Descriptions`: Located in the central treeless zone of the Osogovo mountains, which are situated at the border between south-western Bulgaria and the north-eastern former Yugoslav Republic of Macedonia.

In [17]:
# Article 4
# order_article('58d29193cf58f14928755ba5')

In [18]:
# Try to see the exact location. Wont show last few words.
# pd.DataFrame(order_article('58d29193cf58f14928755ba5').iloc[4])

Article 05  
Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co
* `Site Name`:  Cottonwood III, Cottonwood IV, Red Lady, Red Well, Splains
* `Location`:  38°49'50' 106°24'45', 38°49'50' 106°24'45', 38°52'50' 107°2'30', 38°52'50' 107°2'30',38°53'40' 107°3'15', 38°52'28' 107°05', 38°50' 107°4'30', 38°50' 107°4'30', 38°52' 107°2'30', 38°45' 106°50', 38°45' 106°50'
* `Age Span`:   17000 - 400
* `Site Descriptions`:   The lower timberline or lower forest border is the lower elevational limit of a continuous forest belt. In the Rocky Mountains it forms a sharp contrast with shrublands or grasslands in valley bottoms. Trees may extend below the lower timberline in favorable habitats, along streams or on rocky outcrops.  

In [19]:
# Article 5
# order_article('57928e07cf58f133d1c26609')

In [20]:
# Print the exact line where locations are.
# pd.DataFrame(order_article('57928e07cf58f133d1c26609').iloc[179])

## Linking Neotoma DB to extract locations

In [21]:
# TODO SCRIPT
neotoma = pd.read_csv("../Do_not_commit_data/data-1590729612420.csv")
neotoma = neotoma[['siteid', 'sitename', 'longitudeeast', 'latitudenorth', 'longitudewest', 'latitudesouth', 'sitedescription', 'doi']]

neotoma.head(3)

,siteid,sitename,longitudeeast,latitudenorth,longitudewest,latitudesouth,sitedescription,doi
0,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m...",10.1177/0959683611400199
1,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m...",10.1177/0959683611400199
2,1729,Myrtle Lake,-93.37853,47.98645,-93.39207,47.97876,Lake surrounded by peatland. Physiography: Red...,10.1139/b68-190


In [22]:
nlp_bib = nlp_sentences.merge(bibliography, on = '_gddid')
nlp_bib = nlp_bib.rename(columns={'_id': 'doi'})
nlp_bib.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified,words_as_string,...,title,journal.name.name,author,year,number,volumne,type,pages,link_url,link_type
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['Available', 'online', 'at', 'www.sciencedire...","['JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'CD', ...","['O', 'O', 'O', 'O', 'O', 'O', 'NUMBER', 'O', ...","['available', 'online', 'at', 'www.sciencedire...","['dep', 'dep', 'dep', 'dep', 'dep', 'dep', 'de...","[218, 218, 218, 218, 218, 218, 218, 0, 218, 0,...","Available,online,at,www.sciencedirect.com,Quat...",...,Development of the mixed conifer forest in nor...,Quaternary Research,"Anderson, R. Scott; Jass, Renata B.; Toney, Ja...",2008,2,NaN,article,263--275,http://www.sciencedirect.com/science/article/p...,publisher
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","['The', 'Chihuahueños', 'Bog', 'record', 'exte...","['DT', 'NNP', 'NN', 'NN', 'VBZ', 'TO', 'IN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'NUMBER', ...","['the', 'Chihuahueños', 'bog', 'record', 'exte...","['det', 'compound', 'compound', 'nsubj', '', '...","[4, 4, 4, 5, 0, 11, 11, 11, 11, 11, 5, 0]","The,Chihuahueños,Bog,record,extends,to,over,15...",...,Development of the mixed conifer forest in nor...,Quaternary Research,"Anderson, R. Scott; Jass, Renata B.; Toney, Ja...",2008,2,NaN,article,263--275,http://www.sciencedirect.com/science/article/p...,publisher
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['An', 'Artemisia', 'steppe', ',', 'then', 'an...","['DT', 'NNP', 'NN', ',', 'RB', 'DT', 'JJ', 'NN...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION'...","['a', 'Artemisia', 'steppe', ',', 'then', 'a',...","['det', 'compound', 'nsubj', '', 'advmod', 'de...","[3, 3, 10, 0, 9, 9, 9, 9, 10, 0, 14, 14, 14, 1...","An,Artemisia,steppe,,,then,an,open,Picea,woodl...",...,Development of the mixed conifer forest in nor...,Quaternary Research,"Anderson, R. Scott; Jass, Renata B.; Toney, Ja...",2008,2,NaN,article,263--275,http://www.sciencedirect.com/science/article/p...,publisher


In [23]:
nlp_bib[['words_as_string', 'journal.name.name', 'dms_regex', 'dd_regex']].head(3)

,words_as_string,journal.name.name,dms_regex,dd_regex
0,"Available,online,at,www.sciencedirect.com,Quat...",Quaternary Research,[],[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15...",Quaternary Research,[],[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodl...",Quaternary Research,[],[]


In [24]:
by_doi = neotoma.groupby('doi')
neotoma_sitenames = by_doi.apply(lambda x: list(x['sitename']))
neotoma_siteid = by_doi.apply(lambda x: list(x['siteid']))
neotoma_longeast = by_doi.apply(lambda x: list(x['longitudeeast']))
neotoma_latnorth = by_doi.apply(lambda x: list(x['latitudenorth']))

neotoma_joined_df = neotoma_sitenames.to_frame('sitenames').join(neotoma_siteid.to_frame('siteid')).join(neotoma_longeast.to_frame('longeast')).join(neotoma_latnorth.to_frame('latnorth'))
neotoma_joined_df = neotoma_joined_df.reset_index()

neotoma_joined_df.head(2)

,doi,sitenames,siteid,longeast,latnorth
0,0.1139/e80-122,"[East Baltic Bog, Mermaid Bog, Portage Bog]","[763, 1684, 1903]","[-62.15, -63.025, -64.075]","[46.408333, 46.25, 46.673611]"
1,10.1002/(SICI)1099-1417(199607/08)11:4<327::AI...,[Lundin Tower],[3256],[-2.97206],[56.21529]


From Neotoma DB, we have obtained a dataframe grouped by DOI that lists each `sitename`, `siteid`, `longitude east`, and `latitunde north`. Now, let's join this information to the NLP_Bib dataframe. For now, let's do to the one that coordinates have been obtained with REGEX just to see if some of the numbers and sitenames make sense. 

In [25]:
nlp_bib_neotoma = nlp_bib.merge(neotoma_joined_df, on = 'doi') #, how = 'left')
print(nlp_bib_neotoma.shape)

(106640, 29)


In [26]:
# Filter df to just frames that has REGEX longer than one
mask = ((nlp_bib_neotoma['dms_regex'].str.len() > 0) | (nlp_bib_neotoma['dd_regex'].str.len() > 0))
nlp_bib_neotoma_regex = nlp_bib_neotoma.loc[mask]
nlp_bib_neotoma_regex[['words','dms_regex', 'dd_regex']].head(100)

print(nlp_bib_neotoma_regex.shape)

(106640, 29)


In [27]:
nlp_bib_neotoma_regex = nlp_bib_neotoma_regex[['_gddid', 'sentid', 'doi', 'words', 'words_as_string', 'dms_regex', 'dd_regex', 'sitenames', 'latnorth', 'longeast']]
nlp_bib_neotoma_regex = nlp_bib_neotoma_regex.reset_index()
nlp_bib_neotoma_regex.head(3)

,index,_gddid,sentid,doi,words,words_as_string,dms_regex,dd_regex,sitenames,latnorth,longeast
0,0,54b43266e138239d8684efed,1,10.1016/j.yqres.2007.12.002,"['Available', 'online', 'at', 'www.sciencedire...","Available,online,at,www.sciencedirect.com,Quat...",[],[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"
1,1,54b43266e138239d8684efed,2,10.1016/j.yqres.2007.12.002,"['The', 'Chihuahueños', 'Bog', 'record', 'exte...","The,Chihuahueños,Bog,record,extends,to,over,15...",[],[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"
2,2,54b43266e138239d8684efed,3,10.1016/j.yqres.2007.12.002,"['An', 'Artemisia', 'steppe', ',', 'then', 'an...","An,Artemisia,steppe,,,then,an,open,Picea,woodl...",[],[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"


We can see that most of the REGEX indeed pulled out the same locations for latitude and longitudes as in the Neotoma database.

In [28]:
nlp_bib_neotoma[['_gddid', 'sentid', 'doi', 'words', 'dms_regex', 'sitenames', 'latnorth', 'longeast']].head(2)

,_gddid,sentid,doi,words,dms_regex,sitenames,latnorth,longeast
0,54b43266e138239d8684efed,1,10.1016/j.yqres.2007.12.002,"['Available', 'online', 'at', 'www.sciencedire...",[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"
1,54b43266e138239d8684efed,2,10.1016/j.yqres.2007.12.002,"['The', 'Chihuahueños', 'Bog', 'record', 'exte...",[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"


In [29]:
nlp_bib_neotoma['latnorth_s'] = nlp_bib_neotoma['latnorth'].apply(lambda x: [int(i) for i in x])
nlp_bib_neotoma['latnorth_s'] = nlp_bib_neotoma['latnorth_s'].apply(lambda x: [str(i) for i in x])

nlp_bib_neotoma['longeast_s'] = nlp_bib_neotoma['longeast'].apply(lambda x: [int(abs(i)) for i in x])
nlp_bib_neotoma['longeast_s'] = nlp_bib_neotoma['longeast_s'].apply(lambda x: [str(i) for i in x])
nlp_bib_neotoma['longeast_s'] = nlp_bib_neotoma['longeast_s'].replace('-', '', regex = True)

In [30]:
# nlp_bib_neotoma[['latnorth_s', 'longeast_s']].head(-2)

In [31]:
nlp_bib_neotoma['intersection_words_lat'] = nlp_bib_neotoma[['words','latnorth_s']].apply(lambda x : list(set.intersection(*map(set,list(x)))),axis=1)
nlp_bib_neotoma['intersection_words_long'] = nlp_bib_neotoma[['words','longeast_s']].apply(lambda x : list(set.intersection(*map(set,list(x)))),axis=1)
long_lat_int = nlp_bib_neotoma[(nlp_bib_neotoma['intersection_words_lat'].apply(len) > 0) | (nlp_bib_neotoma['intersection_words_long'].apply(len) > 0 )]


mask = ((long_lat_int['dms_regex'].str.len() > 0) | (long_lat_int['dd_regex'].str.len() > 0))
long_lat_int = long_lat_int.loc[mask]
print(long_lat_int.shape)


(2613, 33)


In [32]:
long_lat_int[['words', 'latnorth_s', 'intersection_words_lat', 'longeast_s', 'intersection_words_long', 'dms_regex',  'dd_regex']].head(3)


,words,latnorth_s,intersection_words_lat,longeast_s,intersection_words_long,dms_regex,dd_regex
2590,"['Ecology', ',', '93', '-LRB-', '8', '-RRB-', ...","[0, 0]",[0],"[89, 89]",[],[],[]
2591,"['A', 'high-resolution', '-LRB-', '2', '--', '...","[0, 0]",[0],"[89, 89]",[],[],[]
2596,"['A', 'marked', 'decline', 'in', 'long-term', ...","[0, 0]",[0],"[89, 89]",[],[],[]


In [33]:
path = r'/Users/seiryu8808/Desktop/UWinsc/Github/UnacquiredSites/src/output'
output_file = os.path.join(path,'long_lat_regex_neotoma_int.csv')
long_lat_int.to_csv(output_file, sep='\t', index = False)

## Intersections Between Sitenames and NLP Sentences
Let's now extract Sitenames from the Neotoma database. For this, I will group by doi and then, list all sitenames that are mentioned in each doi.

In [34]:
nlp_bib_neotoma['words_l']  = nlp_bib_neotoma['words'].astype(str).str.lower().transform(ast.literal_eval)


In [35]:
nlp_bib_neotoma['sitenames_l'] = nlp_bib_neotoma['sitenames'].astype(str).str.lower().transform(ast.literal_eval)
nlp_bib_neotoma[['words_l', 'sitenames_l']].head(2)

,words_l,sitenames_l
0,"[available, online, at, www.sciencedirect.com,...","[chihuahueños bog, chihuahueños bog]"
1,"[the, chihuahueños, bog, record, extends, to, ...","[chihuahueños bog, chihuahueños bog]"


In [36]:
# nlp_bib_neotoma['sitenames_l']

In [37]:
nlp_bib_neotoma['intersection_words_sitenames'] = nlp_bib_neotoma[['words_l','sitenames_l']].apply(lambda x : list(set.intersection(*map(set,list(x)))),axis=1)
#nlp_bib_neotoma['intersection_words_sitenames']

sitenames_intersection = nlp_bib_neotoma[nlp_bib_neotoma['intersection_words_sitenames'].apply(len) > 0 ]
print(sitenames_intersection.shape)
sitenames_intersection[['words_l', 'sitenames_l', 'intersection_words_sitenames', 'dms_regex', 'dd_regex']].head(3)


(438, 36)


,words_l,sitenames_l,intersection_words_sitenames,dms_regex,dd_regex
681,"[index, lat, -lrb-, ◦, n, -rrb-, long, -lrb-, ...","[anderson lake, anderson lake, bayers, bayers,...",[bayers],[],[]
788,"[seven, lakes, -lrb-, bayers, ,, bell, ,, cran...","[anderson lake, anderson lake, bayers, bayers,...",[bayers],[],[]
2110,"[quaternary, research, 16, ,, 66-79, -lrb-, 19...","[norwood, norwood, norwood]",[norwood],[],[]


In [38]:
output_file = os.path.join(path,'sitenames_int.csv')
sitenames_intersection.to_csv(output_file, sep='\t', index = False)